In [2]:
import json
import boto3
import os

In [3]:
def send_survey(survey_path, sqs_url):
    '''
        Input:
            survey_path (str): path to JSON survey data (e.g. ‘./survey.json’)
            sqs_url (str): URL for SQS queue
        Output:
            StatusCode (int): indicating whether the survey was successfully 
            sent into the SQS queue (200) or not (0)
    '''
    # Load survey data from the file
    with open(survey_path, 'r') as file:
        survey_data = json.load(file)

    # Create a boto3 client for SQS
    sqs_client = boto3.client('sqs')

    # Send the message to the SQS queue
    try:
        response = sqs_client.send_message(
            QueueUrl=sqs_url,
            MessageBody=json.dumps(survey_data)
        )
        return 200  # Successful status code
    except Exception as e:
        print(f"Failed to send message: {e}")
        return 0  # Failure status code

In [19]:
lambda_client = boto3.client('lambda')
iam_client = boto3.client('iam')

# Initialize S3 client
s3_client = boto3.client('s3')
# Define the bucket name (must be globally unique)
bucket_name = 'a2q1-bucket'

try:
    # Create the S3 bucket
    s3_client.create_bucket(Bucket=bucket_name)
    print(f"S3 bucket {bucket_name} created successfully.")
except Exception as e:
    print(f"Error creating S3 bucket {bucket_name}: {str(e)}")


# Attempt to retrieve the IAM role
try:
    role = iam_client.get_role(RoleName='LabRole')
    role_arn = role['Role']['Arn']
except iam_client.exceptions.NoSuchEntityException:
    print("Specified IAM role does not exist.")
    raise
except Exception as e:
    print(f"Error retrieving IAM role: {str(e)}")
    raise

# Initialize a DynamoDB client
dynamodb = boto3.resource('dynamodb')
dynamodb_client = boto3.client('dynamodb')

# Define the table name
table_name = 'a2q1-table'

try: 
    # Create the DynamoDB table
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {'AttributeName': 'user_id', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'timestamp', 'KeyType': 'RANGE'}  # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'user_id', 'AttributeType': 'S'},
            {'AttributeName': 'timestamp', 'AttributeType': 'S'}
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 1,
            'WriteCapacityUnits': 1
        }
    )
    table.meta.client.get_waiter('table_exists').wait(TableName=table_name)

    print(f"DynamoDB table {table_name} created successfully.")
except Exception as e:
    print(f"Error creating DynamoDB table: {str(e)}")


S3 bucket a2q1-bucket created successfully.
Error creating DynamoDB table: An error occurred (ResourceInUseException) when calling the CreateTable operation: Table already exists: a2q1-table


In [5]:
with open('q1c_lambda.zip', 'rb') as f:
    lambda_zip = f.read()

try:
    response = lambda_client.create_function(
        FunctionName='a2q1_lambda',
        Runtime='python3.11',
        Role=role['Role']['Arn'],
        Handler='q1c_lambda.lambda_handler',
        Code=dict(ZipFile=lambda_zip),
        Timeout=300
)
    print(response)
except lambda_client.exceptions.ResourceConflictException:
    response = lambda_client.update_function_code(
        FunctionName='a2q1_lambda',
        ZipFile=lambda_zip
        )
    print(response)
except Exception as e:
    print(f"Error creating or updating Lambda function: {str(e)}")


try:
    response = lambda_client.put_function_concurrency(
        FunctionName='a2q1_lambda',
        ReservedConcurrentExecutions=10
    )
    print(response)
except Exception as e:
    print(f"Error setting concurrency for Lambda function: {str(e)}")

{'ResponseMetadata': {'RequestId': 'e0b1c859-e841-4e75-97b2-2b285cb38080', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 02 May 2024 14:04:01 GMT', 'content-type': 'application/json', 'content-length': '1316', 'connection': 'keep-alive', 'x-amzn-requestid': 'e0b1c859-e841-4e75-97b2-2b285cb38080'}, 'RetryAttempts': 0}, 'FunctionName': 'a2q1_lambda', 'FunctionArn': 'arn:aws:lambda:us-east-1:102168828713:function:a2q1_lambda', 'Runtime': 'python3.11', 'Role': 'arn:aws:iam::102168828713:role/LabRole', 'Handler': 'q1c_lambda.lambda_handler', 'CodeSize': 1677, 'Description': '', 'Timeout': 300, 'MemorySize': 128, 'LastModified': '2024-05-02T14:04:01.000+0000', 'CodeSha256': 'xSWg7zlvQlWsS90yejTp8n63tKEPF9/mFPzldQYSbuY=', 'Version': '$LATEST', 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': '818d6eef-07d1-4e3d-81c7-c59ff14e1824', 'State': 'Active', 'LastUpdateStatus': 'InProgress', 'LastUpdateStatusReason': 'The function is being created.', 'LastUpdateStatusReasonCode': 'Creati

In [ ]:
'''
import boto3

# Create an SQS client
sqs_client = boto3.client('sqs')

# Specify the name of the queue
queue_name = 'a2q1-queue'

# Get the URL of the queue
queue_url_response = sqs_client.get_queue_url(QueueName=queue_name)
queue_url = queue_url_response['QueueUrl']

# Get the queue ARN using the queue URL
queue_attributes_response = sqs_client.get_queue_attributes(
    QueueUrl=queue_url,
    AttributeNames=['QueueArn']
)
queue_arn = queue_attributes_response['Attributes']['QueueArn']

sqs_client.set_queue_attributes(
    QueueUrl=queue_url,
    Attributes={'VisibilityTimeout': '1800'}
)

response = lambda_client.create_event_source_mapping(
    FunctionName='a2q1_lambda',
    EventSourceArn=queue_arn,
    Enabled=True,
    BatchSize=10
)
'''

In [13]:
import boto3

# Create an SQS client
sqs_client = boto3.client('sqs')
lambda_client = boto3.client('lambda')  # Ensure lambda_client is defined

# Specify the name of the queue
queue_name = 'a2q1-queue'


try:
    response = sqs_client.create_queue(
        QueueName=queue_name
    )
    print("Queue created successfully:", response['QueueUrl'])
except sqs_client.exceptions.QueueNameExists:
    print("Queue with the same name already exists.")
except Exception as e:
    print("An error occurred:", e)


try:
    # Get the URL of the queue
    queue_url_response = sqs_client.get_queue_url(QueueName=queue_name)
    queue_url = queue_url_response['QueueUrl']

    # Get the queue ARN using the queue URL
    queue_attributes_response = sqs_client.get_queue_attributes(
        QueueUrl=queue_url,
        AttributeNames=['QueueArn']
    )
    queue_arn = queue_attributes_response['Attributes']['QueueArn']

    # Set queue attributes
    sqs_client.set_queue_attributes(
        QueueUrl=queue_url,
        Attributes={'VisibilityTimeout': '1800'}
    )
    print("Visibility timeout set successfully.")

    # Create event source mapping between Lambda and the SQS queue
    response = lambda_client.create_event_source_mapping(
        FunctionName='a2q1_lambda',
        EventSourceArn=queue_arn,
        Enabled=True,
        BatchSize=10
    )
    print("Event source mapping created successfully:", response)

except Exception as e:
    print(f"An error occurred: {str(e)}")

Queue created successfully: https://sqs.us-east-1.amazonaws.com/102168828713/a2q1-queue
Visibility timeout set successfully.
An error occurred: An error occurred (InvalidParameterValueException) when calling the CreateEventSourceMapping operation: Function does not exist


In [ ]:
'''
json_directory = './test_json'

# Iterate over each file in the directory and send it to the SQS queue
for filename in os.listdir(json_directory):
    if filename.endswith('.json'):
        file_path = os.path.join(json_directory, filename)
        status_code = send_survey(file_path, queue_url)
        print(f"File: {filename}, Status Code: {status_code}")
'''


In [7]:
json_directory = './test_json'

# Check if the directory exists
if not os.path.exists(json_directory):
    print(f"Directory not found: {json_directory}")
else:
    # Iterate over each file in the directory and send it to the SQS queue
    for filename in os.listdir(json_directory):
        if filename.endswith('.json'):
            file_path = os.path.join(json_directory, filename)
            status_code = send_survey(file_path, queue_url)
            print(f"File: {filename}, Status Code: {status_code}")

File: test1.json, Status Code: 200
File: test6.json, Status Code: 200
File: test7.json, Status Code: 200
File: test8.json, Status Code: 200
File: test4.json, Status Code: 200
File: test5.json, Status Code: 200
File: test9.json, Status Code: 200
File: test10.json, Status Code: 200
File: test2.json, Status Code: 200
File: test3.json, Status Code: 200


In [ ]:
'''
# Delete the SQS queue
response = sqs_client.delete_queue(QueueUrl=queue_url)
print(response)

# Specify your bucket and table names

# Call the functions to delete the resources
s3_client.delete_s3_bucket(bucket_name)
sqs_client.delete_dynamodb_table(table_name)
lambda_client.delete_function(FunctionName=FunctionName)
'''

In [20]:
# Delete the SQS queue
try:
    sqs_response = sqs_client.delete_queue(QueueUrl=queue_url)
    print("SQS Queue deleted successfully:", sqs_response)
except Exception as e:
    print("Failed to delete SQS Queue:", str(e))

# Delete the S3 bucket and all its contents
try:
    bucket_objects = s3_client.list_objects_v2(Bucket=bucket_name)
    if 'Contents' in bucket_objects:
        for obj in bucket_objects['Contents']:
            s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
    s3_client.delete_bucket(Bucket=bucket_name)
    print("S3 Bucket deleted successfully")
except Exception as e:
    print("Failed to delete S3 Bucket:", str(e))

# Delete the DynamoDB table
try:
    dynamodb_client.delete_table(TableName=table_name)
    print("DynamoDB Table deleted successfully")
except Exception as e:
    print("Failed to delete DynamoDB Table:", str(e))

# Delete the Lambda function
try:
    lambda_client.delete_function(FunctionName='a2q1_lambda')
    print("Lambda Function deleted successfully")
except Exception as e:
    print("Failed to delete Lambda Function:", str(e))

Failed to delete SQS Queue: An error occurred (AWS.SimpleQueueService.NonExistentQueue) when calling the DeleteQueue operation: The specified queue does not exist.
S3 Bucket deleted successfully
DynamoDB Table deleted successfully
Failed to delete Lambda Function: An error occurred (ResourceNotFoundException) when calling the DeleteFunction operation: Function not found: arn:aws:lambda:us-east-1:102168828713:function:a2q1_lambda
